In [ ]:
# Specialist Finders

%matplotlib inline

import os

import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
import theano


def float32(k):
    return np.cast['float32'](k)

class AdjustVariable(object):
    def __init__(self, name, start=0.03, stop=0.001):
        self.name = name
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = float32(self.ls[epoch - 1])
        getattr(nn, self.name).set_value(new_value)




def load(test=False, cols=None):
    FTRAIN = '/home/ec2-user/W207FinalProject/Project/Data/testAugBig2.csv'    
    #FTRAIN = '/home/ec2-user/W207FinalProject/Project/Data/training.csv'
    FTEST = '/home/ec2-user/W207FinalProject/Project/Data/test.csv'

    """Loads data from FTEST if *test* is True, otherwise from FTRAIN.
    Pass a list of *cols* if you're only interested in a subset of the
    target columns.
    """
    fname = FTEST if test else FTRAIN
    #df = read_csv(os.path.expanduser(fname))  # load pandas dataframe
    df = read_csv(fname)  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    if cols:  # get a subset of columns
        df = df[list(cols) + ['Image']]

    print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255.  # scale pixel values to [0, 1]
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = y.astype(np.float32)
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        
    else:
        y = None

    return X, y


#X, y = load()
#print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(
#    X.shape, X.min(), X.max()))
#print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(
#    y.shape, y.min(), y.max()))

def load2d(test=False, cols=None):
    X, y = load(test=test,cols=cols)
    X = X.reshape(-1, 1, 96, 96)
    return X, y

############################################################################################
############################################################################################

class SpecialistNet(object):

    def __init__(self,col_num):
        
        self.train_X, self.train_y = load2d(test = False, cols = col_num)
        self.build_net()
        self.train_net()
        
    def build_net(self):
        self.thisNet = NeuralNet(
            layers=[
                ('input', layers.InputLayer),
                ('conv1', layers.Conv2DLayer),
                ('pool1', layers.MaxPool2DLayer),
                ('conv2', layers.Conv2DLayer),
                ('pool2', layers.MaxPool2DLayer),
                ('conv3', layers.Conv2DLayer),
                ('pool3', layers.MaxPool2DLayer),
                ('hidden4', layers.DenseLayer),
                ('hidden5', layers.DenseLayer),
                ('hidden6', layers.DenseLayer),
                ('output', layers.DenseLayer),
                ],

            input_shape=(None, 1, 96, 96),
            conv1_num_filters=16, conv1_filter_size=(4, 4), pool1_pool_size=(2, 2),
            conv2_num_filters=24, conv2_filter_size=(3, 3), pool2_pool_size=(2, 2),
            conv3_num_filters=32, conv3_filter_size=(2, 2), pool3_pool_size=(2, 2),
            hidden4_num_units=500, hidden5_num_units=100, hidden6_num_units=30,
            output_num_units=1, output_nonlinearity=None,

            update_learning_rate=theano.shared(float32(0.1)),
            update_momentum=theano.shared(float32(0.9)),

            regression=True,
            #batch_iterator_train = WonkyBatchIterator(batch_size=128),

            on_epoch_finished=[
            AdjustVariable('update_learning_rate', start=0.1, stop=0.0001),
            AdjustVariable('update_momentum', start=0.9, stop=0.999),],

            max_epochs=250,
            verbose=1,
            )

    def train_net(self):

        self.thisNet.fit(self.train_X, self.train_y)

    def predict(self):
        test_X = load2d(test=True)
        self.predictions = self.thisNet.predict(test_X)

        return self.predictions

        
############################################################################################
############################################################################################
colnames = [u'left_eye_center_x', u'left_eye_center_y', u'right_eye_center_x',
       u'right_eye_center_y', u'left_eye_inner_corner_x',
       u'left_eye_inner_corner_y', u'left_eye_outer_corner_x',
       u'left_eye_outer_corner_y', u'right_eye_inner_corner_x',
       u'right_eye_inner_corner_y', u'right_eye_outer_corner_x',
       u'right_eye_outer_corner_y', u'left_eyebrow_inner_end_x',
       u'left_eyebrow_inner_end_y', u'left_eyebrow_outer_end_x',
       u'left_eyebrow_outer_end_y', u'right_eyebrow_inner_end_x',
       u'right_eyebrow_inner_end_y', u'right_eyebrow_outer_end_x',
       u'right_eyebrow_outer_end_y', u'nose_tip_x', u'nose_tip_y',
       u'mouth_left_corner_x', u'mouth_left_corner_y', u'mouth_right_corner_x',
       u'mouth_right_corner_y', u'mouth_center_top_lip_x',
       u'mouth_center_top_lip_y', u'mouth_center_bottom_lip_x',
       u'mouth_center_bottom_lip_y']

allNets = []
for col in colnames:
    allNets.append(SpecialistNet([col]))

import cPickle as pickle
import datetime as dt
now = str(dt.datetime.now()).replace(' ','_').split('.')[0]
with open('/mnt/Specialized_CNN2' + now + '.pickle', 'wb') as f:
    pickle.dump(allNets, f, -1)

#xx,yy = load2d(test=False,cols=[colnames[1]])

Using gpu device 0: GRID K520 (CNMeM is disabled, CuDNN not available)
/usr/local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


left_eye_center_x    5000
Image                5000
dtype: int64
# Neural Network with 1660517 learnable parameters

## Layer information

  #  name     size
---  -------  --------
  0  input    1x96x96
  1  conv1    16x93x93
  2  pool1    16x46x46
  3  conv2    24x44x44
  4  pool2    24x22x22
  5  conv3    32x21x21
  6  pool3    32x10x10
  7  hidden4  500
  8  hidden5  100
  9  hidden6  30
 10  output   1

  epoch    train loss    valid loss    train/val  dur
-------  ------------  ------------  -----------  -----
      1       0.07506       0.00331     22.69057  5.99s
      2       0.00341       0.00329      1.03513  5.98s
      3       0.00341       0.00329      1.03507  5.99s
      4       0.00341       0.00329      1.03550  5.98s
      5       0.00341       0.00329      1.03581  6.00s
      6       0.00341       0.00329      1.03647  5.98s
      7       0.00341       0.00328      1.03660  5.97s
      8       0.00340       0.00328      1.03696  5.98s
      9       0.00340       0.0